In [1]:



try:
    isMaster
except NameError:
    print('isMaster:', False)
    from app.dotenv import base_dir, data_dir
    from app.change_detect.background_baseline.background_subtractors import apply_bg_subtraction,save_background_model,methods
    import os
    print(f'possible background substraction model: {methods.keys()}')


    video_fname = 'vtest.avi'
    save_path = f'{data_dir}\{video_fname.split(".")[0]}'
    
    # List all image files in both folders (can be the same folder)
    folder_train_add = save_path
    folder_evaluate_add = folder_train_add

    
background_models_add = f'{folder_train_add}/background_models'  # Folder to save background models
os.makedirs(background_models_add, exist_ok=True)



baseModel = 'MOG2'


isMaster: False
possible background substraction model: dict_keys(['MOG2', 'KNN', 'CNT', 'LSBP', 'GSOC', 'PBAS'])


In [2]:

if not  os.path.exists( os.path.join(background_models_add, f'background_model_{baseModel}.png')) :
    print(f"Processing with {baseModel}")

    # 1. Calculate the background model from images in image_files1
    image_files_train = sorted([f"{folder_train_add}/{f}" for f in os.listdir(folder_train_add) if f.endswith(('.jpg', '.png'))]) # images for background modeling (training)
    fgbg = apply_bg_subtraction(methods[baseModel], image_files_train)
    print(f"{baseModel} background model calculation complete.")

    # 2. Save the background model as an image - skip if method saving fails 
    if save_background_model(baseModel, fgbg, save_dir = background_models_add):
        print(f"{baseModel} background model saved.")


    # # 3. Apply the background model to images in image_files2 and display the foreground subtraction
    # image_files_evaluate = sorted([f"{folder_evaluate_add}/{f}" for f in os.listdir(folder_evaluate_add) if f.endswith(('.jpg', '.png'))]) # images for foreground subtraction (testing)
    # _ = apply_bg_subtraction(fgbg, image_files_evaluate, substract=True)
    # # Clean up
    # cv.destroyAllWindows()


print(f"{baseModel} background substraction complete.")



MOG2 background substraction complete.


check diffrent models

In [3]:
# fitted_fgbg ={}
# for method_name, fgbg in methods.items():
#     print(f"Processing with {method_name}")

#     # 1. Calculate the background model from images in image_files1
#     ffgbg = apply_bg_subtraction(fgbg, image_files1,LEN=10)
#     print(f"{method_name} background model calculation complete.")
    
#     # 2. Save the background model as an image - skip if method saving fails 
#     if save_background_model(method_name, fgbg, save_dir = background_models_add):
#         print(f"{method_name} background model saved.")
#         fitted_fgbg[method_name] = fgbg

#     # 3. Apply the background model to images in image_files2 and display the foreground subtraction
#     _ = apply_bg_subtraction(fgbg, image_files2, substract=True,LEN=10)
#     # Clean up
#     cv.destroyAllWindows()
#     print(f"{method_name} background substraction complete.")
    
#     print('-'*50)





# print(f'models saved into images: {fitted_fgbg.keys()}')

hybrid deep learning model

In [4]:
# import cv2
# import torch
# import torchvision.transforms as T
# from torchvision.models.segmentation import deeplabv3_resnet50
# from segmentation_models_pytorch import Unet
# import numpy as np
# import matplotlib.pyplot as plt
# from app.dotenv import base_dir, data_dir

# video_fname = 'vtest.avi'
# video_path = f'{data_dir}\{video_fname}'

# # Load the pre-trained DeepLabV3 model
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# def get_model(model_name):
#     """Get the specified segmentation model
#     Args: model_name (str): Name of the model to load ('DeepLabV3' or 'Unet')
#     Returns: model: Segmentation model
#              transform: Image transformation function
    
#     """
#     print(f"Loading {model_name} model...")
#     if model_name == 'DeepLabV3':
#         model = deeplabv3_resnet50(pretrained=True)
#     elif model_name == 'Unet':
#         model = Unet(
#             encoder_name="resnet34",  # You can change this to resnet50, etc.
#             encoder_weights="imagenet",  # Use ImageNet pre-trained weights
#             in_channels=3,  # RGB image input
#             classes=21,  # Number of output classes (adjust as needed for your segmentation task)
#         )
#     else:
#         raise ValueError(f"Invalid model name: {model_name}")
    
#     # Define transformation for input images
#     # The mean and standard deviation (std) values are computed for the dataset on which the model is pretrained. 
#     # For ImageNet (for R, G, B channels), these values are:  
#     meanImageNet=[0.485, 0.456, 0.406]
#     stdImageNet=[0.229, 0.224, 0.225]
#     transform = T.Compose([
#         T.ToPILImage(),
#         T.Resize((256, 256)),
#         T.ToTensor(),
#         T.Normalize(mean=meanImageNet, std=stdImageNet)
#     ])
#     return model,transform
    

# model,transform = get_model('Unet')
# model = model.eval().to(device)






# def mog2_background_subtraction(frame, mog2):
#     """Apply MOG2 Background Subtraction."""
#     fg_mask = mog2.apply(frame)
#     return fg_mask

# def deep_learning_refinement(frame, model):
#     """Refine MOG2 mask with deep learning-based segmentation."""
#     input_tensor = transform(frame).unsqueeze(0).to(device)
#     with torch.no_grad():
#         output = model(input_tensor)
#     output = output[0] if type(output) == torch.Tensor else output['out'][0] # Output segmentation map: torch.Size([21, 256, 256])
#     segmentation_map = output.argmax(0).byte().cpu().numpy()
#     return segmentation_map

# def deep_learning_refinement(frame, model):
#     """Refine MOG2 mask with deep learning-based segmentation."""
    
#     # Transform the frame (input image) into a format suitable for the model.
#     # The frame is converted into a tensor and resized to the expected input size.
#     # The transformation also normalizes the image using pre-trained model's mean and std.
#     input_tensor = transform(frame).unsqueeze(0).to(device)

#     # Use the model to predict the segmentation map for the input image.
#     with torch.no_grad():
#         output = model(input_tensor) 
#     # Output: tensor of shape [21, 256, 256] where 21 corresponds to the number of classes (including background).
#     output = output[0] if type(output) == torch.Tensor else output['out'][0]  

#     # Apply the argmax operation across the 'channels' (axis 0) to get the most likely class for each pixel.
#     # The model output for each pixel is a vector of 21 class probabilities, and we select the class with the highest probability.
#     segmentation_map = output.argmax(0).byte().cpu().numpy()
    
#     # Explanation of argmax:
#     # The output tensor has shape [21, 256, 256], with each "slice" (along axis 0) representing the 
#     # predicted probability map for one of the 21 classes.
#     # `argmax(0)` selects the index of the class with the highest probability for each pixel.
#     # The result is a 2D map (256, 256) with pixel values representing class indices (0 to 20).

#     return segmentation_map



# # def combine_masks(mog2_mask, dl_mask):
# #     """Combine MOG2 and Deep Learning masks."""
# #     combined = cv2.bitwise_and(mog2_mask, mog2_mask, mask=dl_mask)
# #     return combined

# def combine_masks(mog2_mask, dl_mask):
#     """Combine MOG2 and Deep Learning masks."""
#     # Resize MOG2 mask to match the shape of the DeepLabV3 mask
#     resized_mog2_mask = cv2.resize(mog2_mask, (dl_mask.shape[1], dl_mask.shape[0]))

#     # Combine masks (bitwise AND)
#     combined = cv2.bitwise_and(resized_mog2_mask, resized_mog2_mask, mask=dl_mask)
#     return combined

# # Initialize MOG2
# mog2 = cv2.createBackgroundSubtractorMOG2()

# # Video Capture
# cap = cv2.VideoCapture(video_path)

# # while True:
# #     ret, frame = cap.read()
# #     if not ret:
# #         break

# #     # Step 1: MOG2 Background Subtraction
    
# #     fg_mask = mog2.apply(frame)

# #     # Step 2: Deep Learning-Based Refinement
# #     dl_mask = deep_learning_refinement(frame, model)

# #     # Step 3: Combine Masks
# #     combined_mask = combine_masks(fg_mask, dl_mask)

# #     # Visualization
# #     cv2.imshow("Original Frame", frame)
# #     cv2.imshow("MOG2 Mask", fg_mask)
# #     cv2.imshow("Deep Learning Mask", dl_mask * 255)
# #     cv2.imshow("Combined Mask", combined_mask)

# #     if cv2.waitKey(30) & 0xFF == ord('q'):
# #         break

# # cap.release()
# # cv2.destroyAllWindows()


# import matplotlib.pyplot as plt

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Step 1: MOG2 Background Subtraction
#     fg_mask = mog2.apply(frame)

#     # Step 2: Deep Learning-Based Refinement
#     dl_mask = deep_learning_refinement(frame, model)

#     # Step 3: Combine Masks
#     combined_mask = combine_masks(fg_mask, dl_mask)

#     # Resize masks to match the original frame size for display
#     fg_mask_resized = cv2.resize(fg_mask, (frame.shape[1], frame.shape[0]))
#     dl_mask_resized = cv2.resize(dl_mask * 255, (frame.shape[1], frame.shape[0]))  # Scaling for display
#     combined_mask_resized = cv2.resize(combined_mask, (frame.shape[1], frame.shape[0]))

#     # Create a figure with 1 row and 4 columns (for original frame and three masks)
#     fig, axs = plt.subplots(1, 4, figsize=(15, 5))

#     # Display each image in its own subplot
#     axs[0].imshow(frame)
#     axs[0].set_title("Original Frame")
#     axs[0].axis("off")  # Hide axes

#     axs[1].imshow(fg_mask_resized, cmap='gray')
#     axs[1].set_title("MOG2 Mask")
#     axs[1].axis("off")  # Hide axes

#     axs[2].imshow(dl_mask_resized, cmap='gray')
#     axs[2].set_title("Deep Learning Mask")
#     axs[2].axis("off")  # Hide axes

#     axs[3].imshow(combined_mask_resized, cmap='gray')
#     axs[3].set_title("Combined Mask")
#     axs[3].axis("off")  # Hide axes

#     plt.show()

#     if cv2.waitKey(30) & 0xFF == ord('q'):
#         break
#     break

# cap.release()
# cv2.destroyAllWindows()
